# Übung 05 - Immobilienpreise 

### <span style="color:CornflowerBlue">Speichern Sie die Übung im Ordner 06-Uebung in Ihrem Git.</span>

In dieser Übung sollen Sie Immobilienpreise mit Hilfe von numerischen und kategorischen Merkmalen aus einem unvollständigen Datensatz mit einem Random-Forest-Regressor schätzen. Zusätzlich sollen mit Hilfe von Kreuzvalidierung Anzahl an Entscheidungsbäumen bestimmen, die das beste Ergebnis liefert.


Original Daten: [Iowa, Ames Housing Datensatz](https://ww2.amstat.org/publications/jse/v19n3/decock/AmesHousing.txt)

De Cock, D., 2011. Ames, Iowa: Alternative to the Boston housing data as an end of semester regression project. Journal of Statistics Education, 19(3).

In [1]:
# importieren der Bibliotheken
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score

# plots direkt im notebook anzeigen
%matplotlib inline

a) Lesen Sie den Datensatz ein. (Laden Sie die Daten mit index_col="Id")

In [2]:
df = pd.read_csv("data.csv", index_col="Id")
print(df.head())

    MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
2          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
3          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
4          Lvl    AllPub    Corner  ...        0    NaN   NaN         NaN   
5          Lvl    AllPub       FR2  ... 

b) Prüfen Sie die Daten auf fehlende Werte.

In [3]:
fehlt = df.isnull().sum()

c) Entfernen Sie die Merkmale (Spalten) mit zu vielen fehlenden Werten. (Zum Beispiel: > 500, > 1000)

In [4]:
df_bereinigt1 = df.dropna(axis=1, thresh=len(df) - 500)
df_bereinigt2 = df.dropna(axis=1, thresh=len(df) - 1000)

d) Aufteilen der Daten in Merkmale (X) und Zielvariable (y)

In [5]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

e) Unterteilen Sie den Datensatz in Trainings- und Testdaten. Teilen Sie die Daten im Verhältnis 80% Trainingsdaten und 20% Testdaten auf.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

f) Befüllen Sie die fehlenden Werte der numerischen Merkmale. Wählen sie hierzu eine geeignete Strategie.

Tip: SimpleImputer(strategy=["mean"|"median"|"most_frequent"|"constant"]) aus sklearn.

In [7]:
num_spalten = X_train.select_dtypes(include=[np.number]).columns
imputer_num = SimpleImputer(strategy="mean")

X_train[num_spalten] = imputer_num.fit_transform(X_train[num_spalten])
X_test[num_spalten] = imputer_num.transform(X_test[num_spalten])

g) Befüllen Sie die fehlenden Werte der kategorischen Variablen. Verwenden Sie hierfür den Wert, der am häufigsten in der Spalte vorkommt.

In [8]:
kat_spalten = X_train.select_dtypes(include=[object]).columns
imputer_kat = SimpleImputer(strategy="most_frequent")

X_train[kat_spalten] = imputer_kat.fit_transform(X_train[kat_spalten])
X_test[kat_spalten] = imputer_kat.transform(X_test[kat_spalten])

h) Wenden Sie One-Hot-Encoding auf die kategorischen Merkmale an.

Tip: OneHotEncoder(handle_unknown='ignore', sparse=False) sparse=False gibt eine voll besetzte Matrix zurück.

In [9]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_encoded = encoder.fit_transform(X_train[kat_spalten])
X_test_encoded = encoder.transform(X_test[kat_spalten])

ohe_columns = encoder.get_feature_names_out(kat_spalten)

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=ohe_columns, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=ohe_columns, index=X_test.index)

num_spalten = X_train.select_dtypes(include=[np.number]).columns

X_train_final = pd.concat([X_train[num_spalten], X_train_encoded_df], axis=1)
X_test_final = pd.concat([X_test[num_spalten], X_test_encoded_df], axis=1)

i) Trainieren Sie einen Randomforest Regressor auf den Trainingsdaten um die Immobilienpreise zu schätzen. Gehen Sie dabei folgendermaßen vor:

* Verwenden Sie eine 5-Fold Kreuzvalidierung um die Anzahl der Bäume zu finden, die das beste Ergebnis liefert (GridSearchCV). Probieren Sie die folgenden Konfigurationen aus: z.B. n_estimators: [50, 100, 150, 200]
* Wählen Sie die Konfiguration mit dem besten Ergebnis aus. Führe Sie diese auf den Testdaten aus und berechnen Sie den Mean Squared Error (MAE) sowie den R2_Score.


Tip: n_jobs=<number of Threads> für parallele Ausführung. 

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

estimators =  [50, 100, 150, 200]

best_estimators = None
best_score = -np.inf
best_rf_model = None

for n_estimators in estimators:
    rf = RandomForestRegressor(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    cv_scores = cross_val_score(rf, X_train_final, y_train, cv=5, scoring='neg_mean_squared_error')
    mean_cv_score = np.mean(cv_scores)
    
    # wenn aktueller Score besser, speichern:
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_estimators = n_estimators
        best_rf_model = rf

best_rf_model.fit(X_train_final, y_train)
y_pred = best_rf_model.predict(X_test_final)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)